<a href="https://colab.research.google.com/github/kawa-t/python_google_colaboratory/blob/main/Websc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import tqdm
import xlrd
import pprint
import sqlite3
import re
import mojimoji
import copy

base_path = os.path.join(os.path.dirname)

AttributeError: ignored